In [46]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ndvi-hackathon-data/hacktest.csv
/kaggle/input/ndvi-hackathon-data/hacktrain.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv
/kaggle/input/dataset/hacktest.csv
/kaggle/input/dataset/hacktrain.csv


**Importing Necessory Files**

In [61]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

In [48]:
# Step 1: Load data
train = pd.read_csv("/kaggle/input/ndvi-hackathon-data/hacktrain.csv")
test = pd.read_csv("/kaggle/input/ndvi-hackathon-data/hacktest.csv")
print('Done')

Done


In [62]:
# Step 3: Inspect columns
print("Train shape:", train.shape)
print("Test shape:", test.shape)

Train shape: (8000, 30)
Test shape: (2845, 29)


In [64]:
# Step 4: Basic Preprocessing
train.fillna(method="ffill", inplace=True)
train.fillna(method="bfill", inplace=True)
test.fillna(method="ffill", inplace=True)
test.fillna(method="bfill", inplace=True)

/tmp/ipykernel_35/3091672494.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train.fillna(method="ffill", inplace=True)
/tmp/ipykernel_35/3091672494.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train.fillna(method="bfill", inplace=True)
/tmp/ipykernel_35/3091672494.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test.fillna(method="ffill", inplace=True)
/tmp/ipykernel_35/3091672494.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test.fillna(method="bfill", inplace=True)


In [67]:
# Step 5: Feature Engineering – NDVI aggregation
ndvi_cols = [col for col in train.columns if '_N' in col]

def add_ndvi_features(df):
    df['ndvi_mean'] = df[ndvi_cols].mean(axis=1)
    df['ndvi_std'] = df[ndvi_cols].std(axis=1)
    df['ndvi_max'] = df[ndvi_cols].max(axis=1)
    df['ndvi_min'] = df[ndvi_cols].min(axis=1)
    return df

train = add_ndvi_features(train)
test = add_ndvi_features(test)
print('Done')

Done


In [69]:
# Step 6: Prepare data
X = train[ndvi_cols + ['ndvi_mean', 'ndvi_std', 'ndvi_max', 'ndvi_min']]
y = train['class']

test_ids = test['ID']
X_test = test[ndvi_cols + ['ndvi_mean', 'ndvi_std', 'ndvi_max', 'ndvi_min']]

In [72]:
# Step 7: Encode target labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [74]:
# Step 8: Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

In [75]:
# Step 9: Train Logistic Regression model
model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_scaled, y_encoded)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [77]:
# Step 10: Predict on test set
y_test_pred_encoded = model.predict(X_test_scaled)
y_test_pred = le.inverse_transform(y_test_pred_encoded)

In [78]:
# Step 11: Create submission
submission = pd.DataFrame({
    "ID": test_ids,
    "class": y_test_pred
})
submission.to_csv("submission.csv", index=False)

# Optional: Check the class distribution in submission
print(submission['class'].value_counts())

class
forest        1160
farm           632
impervious     383
grass          341
orchard        178
water          151
Name: count, dtype: int64


In [79]:
from IPython.display import FileLink

FileLink(r'submission.csv')


/kaggle/working/submission.csv